In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.appName("classificationExample1").getOrCreate()

# 타이타닉 데이터를 이용한 생존여부 예측 모델

## 로지스틱 회귀

In [10]:
data = spark.read.csv("learning_spark_data/titanic.csv", header=True, inferSchema=True)
data.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [11]:
# 결측치 처리
from pyspark.sql.functions import col, sum, when, isnan
null_counts = data.select(
                    [
                        sum(when(col(c).isNull() | isnan(c), 1).otherwise(0)).alias(c)
                        for c in data.columns
                    ]
                )
null_counts.show()

+-----------+--------+------+----+------+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Gender|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+------+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|     0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+------+---+-----+-----+------+----+-----+--------+



In [12]:
#feature selection
data_1 = data.select('Survived', 'Pclass', 'Gender', 'Age', 'SibSp', 'Parch', 'Fare')
data_1.show(3)

+--------+------+------+----+-----+-----+-------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|
+--------+------+------+----+-----+-----+-------+
|       0|     3|  male|22.0|    1|    0|   7.25|
|       1|     1|female|38.0|    1|    0|71.2833|
|       1|     3|female|26.0|    0|    0|  7.925|
+--------+------+------+----+-----+-----+-------+
only showing top 3 rows



In [17]:
# age 결특치 처리 - 평균값으로 대체

mean_age = data_1.select('Age').agg({
                    'Age' : 'mean'
                }).collect()[0][0]
mean_age

29.69911764705882

In [18]:
data_1 = data_1.fillna({'Age':mean_age})
data_1.show(10)

+--------+------+------+-----------------+-----+-----+-------+
|Survived|Pclass|Gender|              Age|SibSp|Parch|   Fare|
+--------+------+------+-----------------+-----+-----+-------+
|       0|     3|  male|             22.0|    1|    0|   7.25|
|       1|     1|female|             38.0|    1|    0|71.2833|
|       1|     3|female|             26.0|    0|    0|  7.925|
|       1|     1|female|             35.0|    1|    0|   53.1|
|       0|     3|  male|             35.0|    0|    0|   8.05|
|       0|     3|  male|29.69911764705882|    0|    0| 8.4583|
|       0|     1|  male|             54.0|    0|    0|51.8625|
|       0|     3|  male|              2.0|    3|    1| 21.075|
|       1|     3|female|             27.0|    0|    2|11.1333|
|       1|     2|female|             14.0|    1|    0|30.0708|
+--------+------+------+-----------------+-----+-----+-------+
only showing top 10 rows



In [19]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [23]:
# 데이터 인코딩 StringIndexe
indexer = StringIndexer(inputCol='Gender',outputCol = 'SexIndexer')
data_1 = indexer.fit(data_1).transform(data_1)

IllegalArgumentException: requirement failed: Output column SexIndexer already exists.

In [24]:
data_1.show(5)

+--------+------+------+----+-----+-----+-------+----------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|SexIndexer|
+--------+------+------+----+-----+-----+-------+----------+
|       0|     3|  male|22.0|    1|    0|   7.25|       0.0|
|       1|     1|female|38.0|    1|    0|71.2833|       1.0|
|       1|     3|female|26.0|    0|    0|  7.925|       1.0|
|       1|     1|female|35.0|    1|    0|   53.1|       1.0|
|       0|     3|  male|35.0|    0|    0|   8.05|       0.0|
+--------+------+------+----+-----+-----+-------+----------+
only showing top 5 rows



In [27]:
# FeatureVector 생성
assembler = VectorAssembler(
    inputCols=['Pclass','Age','SibSp','Parch','Fare','SexIndexer'],
    outputCol='features'
)
data_1 = assembler.transform(data_1)
data_1.select('features','Survived').show(5)

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[3.0,22.0,1.0,0.0...|       0|
|[1.0,38.0,1.0,0.0...|       1|
|[3.0,26.0,0.0,0.0...|       1|
|[1.0,35.0,1.0,0.0...|       1|
|[3.0,35.0,0.0,0.0...|       0|
+--------------------+--------+
only showing top 5 rows



In [30]:
# 데이터셋 분할
train_data, test_data = data_1.randomSplit([0.8,0.2], seed=42)
train_data.show(5), test_data.show(5)

+--------+------+------+----+-----+-----+------+----------+--------------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|  Fare|SexIndexer|            features|
+--------+------+------+----+-----+-----+------+----------+--------------------+
|       0|     1|female| 2.0|    1|    2|151.55|       1.0|[1.0,2.0,1.0,2.0,...|
|       0|     1|female|25.0|    1|    2|151.55|       1.0|[1.0,25.0,1.0,2.0...|
|       0|     1|  male|18.0|    1|    0| 108.9|       0.0|[1.0,18.0,1.0,0.0...|
|       0|     1|  male|19.0|    1|    0|  53.1|       0.0|[1.0,19.0,1.0,0.0...|
|       0|     1|  male|19.0|    3|    2| 263.0|       0.0|[1.0,19.0,3.0,2.0...|
+--------+------+------+----+-----+-----+------+----------+--------------------+
only showing top 5 rows

+--------+------+------+-----------------+-----+-----+-------+----------+--------------------+
|Survived|Pclass|Gender|              Age|SibSp|Parch|   Fare|SexIndexer|            features|
+--------+------+------+-----------------+-----+-----+--

(None, None)

In [33]:
# 모델 생성
lr = LogisticRegression(featuresCol='features', labelCol='Survived')
lr_model = lr.fit(train_data)
predic = lr_model.transform(test_data)
predic.select('features','Survived','prediction').show(5)

+--------------------+--------+----------+
|            features|Survived|prediction|
+--------------------+--------+----------+
|[1.0,50.0,0.0,0.0...|       0|       1.0|
|[1.0,21.0,0.0,1.0...|       0|       1.0|
|[1.0,24.0,0.0,0.0...|       0|       1.0|
|[1.0,29.0,0.0,0.0...|       0|       1.0|
|[1.0,29.699117647...|       0|       1.0|
+--------------------+--------+----------+
only showing top 5 rows



In [34]:
predic.select('Survived','prediction').groupBy('Survived','prediction').count().show()

+--------+----------+-----+
|Survived|prediction|count|
+--------+----------+-----+
|       1|       0.0|   16|
|       0|       0.0|   72|
|       1|       1.0|   45|
|       0|       1.0|   12|
+--------+----------+-----+



In [36]:
from pyspark.sql.functions import expr
comp = predic.withColumn('correct', expr('case when Survived = prediction then 1 else 0 end'))
comp.where('correct=0').count()

28

In [39]:
# 틀린 데이터만 필터링
predic.filter(col('Survived') != col('prediction')).show()

+--------+------+------+-----------------+-----+-----+--------+----------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Gender|              Age|SibSp|Parch|    Fare|SexIndexer|            features|       rawPrediction|         probability|prediction|
+--------+------+------+-----------------+-----+-----+--------+----------+--------------------+--------------------+--------------------+----------+
|       0|     1|female|             50.0|    0|    0| 28.7125|       1.0|[1.0,50.0,0.0,0.0...|[-1.9520233772457...|[0.12433289705143...|       1.0|
|       0|     1|  male|             21.0|    0|    1| 77.2875|       0.0|[1.0,21.0,0.0,1.0...|[-0.5063625084573...|[0.37604662481402...|       1.0|
|       0|     1|  male|             24.0|    0|    0|    79.2|       0.0|[1.0,24.0,0.0,0.0...|[-0.5000095386389...|[0.37753842718518...|       1.0|
|       0|     1|  male|             29.0|    0|    0|    30.0|       0.0|[1.0,29.0,0.0,0.0...|[-0.1615540

In [42]:
# 정확도 평가
comp.selectExpr('avg(correct) as accuracy').collect()[0]['accuracy']

0.8068965517241379

In [40]:
eval = BinaryClassificationEvaluator(labelCol='Survived', rawPredictionCol='rawPrediction',metricName='areaUnderROC')
auc = eval.evaluate(predic)
auc

0.8664129586260734

AUROC -> X축 FPR, y축 TPR의 곡선 아래면적, 1에 가까울수록 좋은 모델

In [44]:
spark.stop()

# libsvm 파일 형식의 처리
텍스트파일 형식, 희소데이터 용 압축파일이다.메모리, 처리속도 개선 - 머신러닝에서 활용되는 형식<br>
레이블 행:값 행:값

In [45]:
spark = SparkSession.builder.appName("classificationExample2").getOrCreate()

In [46]:
data2 = spark.read.format('libsvm').load('learning_spark_data/sample_libsvm_data.txt')
data2.count()

100

In [47]:
data2.show(3)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 3 rows



In [48]:
train_data, test_data = data2.randomSplit([0.7,0.3],seed=12)

In [50]:
train_data.count(), test_data.count()

(73, 27)

In [51]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(train_data)
pred = lrModel.transform(test_data)
pred.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[0.95414441393043...|[0.72194788886091...|       0.0|
|  0.0|(692,[100,101,102...|[0.48568919283978...|[0.61909039185191...|       0.0|
|  0.0|(692,[123,124,125...|[1.00961478127974...|[0.73294475454830...|       0.0|
|  0.0|(692,[126,127,128...|[0.90293696106823...|[0.71155267489794...|       0.0|
|  0.0|(692,[126,127,128...|[0.75830339388632...|[0.68098526895593...|       0.0|
|  0.0|(692,[127,128,129...|[0.97194839391749...|[0.72550768363690...|       0.0|
|  0.0|(692,[127,128,129...|[0.88414239930651...|[0.70767989197633...|       0.0|
|  0.0|(692,[127,128,129...|[0.96585465524705...|[0.72429247205628...|       0.0|
|  0.0|(692,[128,129,130...|[0.91063904393044...|[0.71313091310478...|       0.0|
|  0.0|(692,[152

In [54]:
eval = BinaryClassificationEvaluator(metricName='areaUnderROC')
auc = eval.evaluate(pred)
auc

1.0

In [55]:
spark.stop()